# testcell

In [ ]:
#| hide
import testcell

**TL;DR**: `%%testcell` prevents your testing cells from affecting the global namespace.

The Python cell magic `%%testcell` executes a cell without *polluting* the notebook's global namespace. This is useful whenever you want to test your code without having any of the local variables escape that cell.

What's happening under the hood is that your cell code, before being executed, is wrapped in a temporary function that will be deleted after execution. To give you the feeling of *seamless integration* the last statement is optionally returned like it happens in a normal cell.

**WARNING:** this don't protect you from *the side effects of your code* like deleting a file or mutating the state of a global variable.

[![](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/artste/testcell/blob/main/demo/testcell_demo.ipynb) 

[![](https://kaggle.com/static/images/open-in-kaggle.svg)](https://www.kaggle.com/artste/introducing-testcell)

## Install

```sh
pip install testcell
```

## How to use

just import it with `import testcell` and then use the `%%testcell` cell magic.

```python
%%testcell
a = "'a' is not polluting global scope"
a
```

In [ ]:
%%testcell
#| echo: false
a = "'a' is not polluting global scope"
a

"'a' is not polluting global scope"

In [ ]:
assert 'a' not in locals()

What is happening under the hood is that `%%testcell` wraps your cell's code with a function, execute it and then deletes it. Adding the `verbose` keywork will print which code will be executed.

NOTE: The actual cell code is enclosed within `BEGIN` and `END` comment blocks for improved readability.

```python
%%testcell verbose
a = "'a' is not polluting global scope"
a
```

In [ ]:
%%testcell verbose
#| echo: false
a = "'a' is not polluting global scope"
a


### BEGIN
def _test_cell_():
	#| echo: false
	a = "'a' is not polluting global scope"
	return a # %%testcell
try:
	_ = _test_cell_()
finally:
	del _test_cell_
_ # This will be added to global scope
### END


"'a' is not polluting global scope"

If you're just interested in seeing what will be executed, but actually not executing it, you ca use `dryrun` option:

```python
%%testcell dryrun
a = "'a' is not polluting global scope"
a
```

In [ ]:
%%testcell dryrun
#| echo: false
a = "'a' is not polluting global scope"
a


### BEGIN
def _test_cell_():
	#| echo: false
	a = "'a' is not polluting global scope"
	return a # %%testcell
try:
	_ = _test_cell_()
finally:
	del _test_cell_
if _ is not None: display(_)
### END


If you add a semicolon `;` at the end of your last statement no `return` statement is added and nothing is displayed like a normal jupyter cell.

```python
%%testcell verbose
a = "'a' is not polluting global scope"
a;
```

In [ ]:
%%testcell verbose
#| echo: false
a = "'a' is not polluting global scope"
a;


### BEGIN
def _test_cell_():
	#| echo: false
	a = "'a' is not polluting global scope"
	a;
try:
	_ = _test_cell_()
finally:
	del _test_cell_
_ # This will be added to global scope
### END


`testcell` works seamlessly with existing `print` or `display`statements on last line:

```python
%%testcell verbose
a = "'a' is not polluting global scope"
print(a)
```

In [ ]:
%%testcell verbose
#| echo: false
a = "'a' is not polluting global scope"
print(a)


### BEGIN
def _test_cell_():
	#| echo: false
	a = "'a' is not polluting global scope"
	return print(a) # %%testcell
try:
	_ = _test_cell_()
finally:
	del _test_cell_
_ # This will be added to global scope
### END
'a' is not polluting global scope


Moreover, thanks to `ast`, it properly deals with complex situations like comments on the last line and multi lines statements

```python
%%testcell verbose
a = "'a' is not polluting global scope"
(a,
 True)
# this is a comment on last line
```

In [ ]:
%%testcell verbose
#| echo: false
a = "'a' is not polluting global scope"
(a,
 True)
# this is a comment on last line


### BEGIN
def _test_cell_():
	#| echo: false
	a = "'a' is not polluting global scope"
	return (a,
	 True) # %%testcell
try:
	_ = _test_cell_()
finally:
	del _test_cell_
_ # This will be added to global scope
### END


("'a' is not polluting global scope", True)

### Run in isolation

`%%testcelln` is a shortcut for `%%testcell noglobals` and executes the cell in complete isolation from the global scope. 
This is very useful when you want to ensure that global variables or namespaces are not accessible within the cell.

In [ ]:
aaa = 'global variable'

```python
%%testcell
'aaa' in globals()
```

In [ ]:
%%testcell 
#| echo: false
'aaa' in globals()

True

```python
%%testcell noglobals
'aaa' in globals()
```

In [ ]:
%%testcell noglobals
#| echo: false
'aaa' in globals()

False

```python
%%testcelln
'aaa' in globals()
```

In [ ]:
%%testcelln
#| echo: false
'aaa' in globals()

False

```python
%%testcelln
globals().keys()
```

In [ ]:
%%testcelln
#| echo: false
globals().keys()

dict_keys(['__builtins__'])

With `%%testcelln` inside the cell, you'll be able to access only  to `__builtins__` (aka: standard python's functions). **It behaves like a notebook-in-notebook**.

```python
%%testcell
def my_function(x):
    print(aaa) # global variable
    return x

try:
    my_function(123)
except Exception as e:
    print(e)
```

In [ ]:
%%testcell
#| echo: false
def my_function(x):
    print(aaa) # global variable
    return x

try:
    my_function(123)
except Exception as e:
    print(e)

global variable


```python
%%testcelln
def my_function(x):
    print(aaa) # global variable
    return x

try:
    my_function(123)
except Exception as e:
    print(e)
```

In [ ]:
%%testcelln
#| echo: false
def my_function(x):
    print(aaa) # global variable
    return x

try:
    my_function(123)
except Exception as e:
    print(e)

name 'aaa' is not defined


As you can see from this last example, `%%testcelln` helps you to identify that `my_function` refers global variable `aaa`.

**IMPORTANT**: this is *just wrapping your cell*  and so it's still running on your main kernel. If you modify variables that has been created outside of this cell (aka: if you have side effects) this will not protect you.

In [ ]:
aaa

'global variable'

```python
%%testcell 
# WARNING: this will alter the state of global variable:
globals().update({'aaa' : 'modified global variable'});
```

In [ ]:
%%testcell 
#| echo: false
# WARNING: this will alter the state of global variable:
globals().update({'aaa' : 'modified global variable'});

In [ ]:
aaa

'modified global variable'

In [ ]:
del aaa

## Links:

+ PROJECT PAGE: [https://github.com/artste/testcell](https://github.com/artste/testcell)
+ DOCUMENTATION: [https://artste.github.io/testcell](https://artste.github.io/testcell)
+ PYPI: [https://pypi.org/project/testcell](https://pypi.org/project/testcell)
+ COLAB DEMO: [testcell_demo.ipynb](https://colab.research.google.com/github/artste/testcell/blob/main/demo/testcell_demo.ipynb)
+ KAGGLE SAMPLE NOTEBOOK: [https://www.kaggle.com/artste/introducing-testcell](https://www.kaggle.com/artste/introducing-testcell)

## Todo:

+ Install as a plugin to enable it by default like other cell's magic.